In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM
from scipy import stats

In [2]:
shared_control_ssrt = pd.read_csv("/Users/kritiaxh/Documents/PoldrackLab/SharedControl/notebooks/analysis/output/shared_control_metrics_ssrt.csv")
simple_stop_ssrt = pd.read_csv("/Users/kritiaxh/Documents/PoldrackLab/SharedControl/notebooks/analysis/output/simple_stop_metrics.csv")

# Set the first column as the index to use for merging (assuming it's the subject ID)
shared_control_ssrt.columns = ['subject_id'] + list(shared_control_ssrt.columns[1:])
simple_stop_ssrt.columns = ['subject_id'] + list(simple_stop_ssrt.columns[1:])

# Merge by the first (subject_id) column
merged_df = pd.merge(shared_control_ssrt, simple_stop_ssrt[['subject_id', 'ssrt']], on='subject_id', how='left')

# Rename the SSRT column from simple_stop_ssrt to 'simple_stop_ssrt'
merged_df.rename(columns={'ssrt': 'simple_stop_ssrt'}, inplace=True)

# Set the 'subject_id' column as the index
# merged_df.set_index('subject_id', inplace=True)

# Remove the last row, assuming it's the mean row
merged_df = merged_df.iloc[:-1, :]

# Check the merged dataframe
print(merged_df)

   subject_id  non_ai_ssrt   ai_ssrt  ai_ai_ssrt  simple_stop_ssrt
0        s004     0.222370  0.242520    0.255618          0.206134
1        s005     0.219656  0.232827    0.259703          0.160407
2        s006     0.252661  0.256714    0.293884          0.176054
3        s007     0.258807  0.284119    0.338713          0.203817
4        s008     0.241636  0.255048    0.298183          0.183402
5        s009     0.222365  0.259465    0.255665          0.222842
6        s010     0.246101  0.249927    0.257013          0.169163
7        s011     0.213532  0.246637    0.250694          0.186240
8        s012     0.219835  0.241968    0.233972          0.197466
9        s013     0.266719  0.269863    0.342078          0.278431
10       s014     0.251096  0.230776    0.256233          0.217073
11       s015     0.297396  0.380136    0.336576          0.240112
12       s016     0.299043  0.268403    0.430194          0.252519
13       s017     0.214238  0.267408    0.288116          0.18

In [4]:
# Reshape the data for repeated measures ANOVA
data_long = pd.melt(merged_df, id_vars=['subject_id'], value_vars=['non_ai_ssrt', 'ai_ssrt', 'simple_stop_ssrt'],
                    var_name='condition', value_name='SSRT')

print(data_long)

# Run the repeated measures ANOVA
aovrm = AnovaRM(data_long, 'SSRT', 'subject_id', within=['condition'])
anova_results = aovrm.fit()
print(anova_results)

    subject_id         condition      SSRT
0         s004       non_ai_ssrt  0.222370
1         s005       non_ai_ssrt  0.219656
2         s006       non_ai_ssrt  0.252661
3         s007       non_ai_ssrt  0.258807
4         s008       non_ai_ssrt  0.241636
..         ...               ...       ...
115       s039  simple_stop_ssrt  0.215070
116       s040  simple_stop_ssrt  0.222738
117       s041  simple_stop_ssrt  0.282436
118       s042  simple_stop_ssrt  0.199412
119       s043  simple_stop_ssrt  0.248984

[120 rows x 3 columns]
                 Anova
          F Value Num DF  Den DF Pr > F
---------------------------------------
condition 51.7054 2.0000 78.0000 0.0000



In [6]:
# Conduct the paired t-tests for planned comparisons

# Comparison 1: AI Stop vs non-AI Stop
ai_vs_non_ai = stats.ttest_rel(merged_df['ai_ssrt'], merged_df['non_ai_ssrt'])
print(f"AI Stop vs Non-AI Stop SSRT Comparison: t-statistic = {ai_vs_non_ai.statistic}, p-value = {ai_vs_non_ai.pvalue}")

# Comparison 2: Non-AI vs Simple Stopping
non_ai_vs_simple = stats.ttest_rel(merged_df['non_ai_ssrt'], merged_df['simple_stop_ssrt'])
print(f"Non-AI vs Simple Stopping SSRT Comparison: t-statistic = {non_ai_vs_simple.statistic}, p-value = {non_ai_vs_simple.pvalue}")

AI Stop vs Non-AI Stop SSRT Comparison: t-statistic = 5.616398897831615, p-value = 1.765719683873891e-06
Non-AI vs Simple Stopping SSRT Comparison: t-statistic = 6.362698988629167, p-value = 1.624591647785976e-07


In [7]:
import subprocess
import re

# Run pip freeze and capture output
result = subprocess.run(['pip', 'freeze'], capture_output=True, text=True)

# Use a regex to remove lines that include 'file:///'
cleaned_output = re.sub(r' @ file://[^\n]+', '', result.stdout)

# Write the cleaned output to a new requirements.txt file
with open('requirements.txt', 'w') as f:
    f.write(cleaned_output)

print("Cleaned requirements.txt has been generated.")

Cleaned requirements.txt has been generated.
